#### Задание 1

**Чего хочет менеджер от рекомендательной системы?**

Рост прибыли на определенный процент за определенный промежуток времени

**А у нас вообще есть e-mail-ы пользователей? Для скольки %? Не устарели ли они?**

У пользователя нужно периодически уточнять актуальность его e-mail и номера телефона

**Будем ли использовать СМС и push-уведомления в приложении? Может, будем печатать рекомендации на чеке после оплаты на кассе?**

Думаю будет полезно всё из этого

**Как будет выглядеть e-mail? (решаем задачу топ-10 рекомендаций или ранжирования? И топ-10 ли?)**

Думаю топ-5 достаточно, больше сложнее воспринемаеется в письме, можно оставить ссылку на сайт где мож



In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

In [19]:
def precision(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

In [20]:
def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

In [21]:
def recall(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)

In [22]:
def recall_at_k(recommended_list, bought_list, k=5):
    return recall(recommended_list[:k], bought_list)

In [23]:
data = pd.read_csv('/content/transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(10)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
5,2375,26984851516,1,826249,2,1.98,364,-0.60,1642,1,0.0,0.0
6,2375,26984851516,1,1043142,1,1.57,364,-0.68,1642,1,0.0,0.0
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0
8,2375,26984851516,1,1102651,1,1.89,364,0.00,1642,1,0.0,0.0
9,2375,26984851516,1,6423775,1,2.00,364,-0.79,1642,1,0.0,0.0


In [24]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870..."


In [25]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [26]:
# Заведем фиктивный item_id

data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999_999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix)

user_item_matrix.head(3)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15926885,15926886,15926887,15926927,15927033,15927403,15927661,15927850,16809471,17105257
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
sparse_user_item

<2500x5001 sparse matrix of type '<class 'numpy.float64'>'
	with 709529 stored elements in Compressed Sparse Row format>

In [28]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [29]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

In [30]:
%%time

model = AlternatingLeastSquares(factors=64, 
                                regularization=0.05,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=10,
                                use_gpu=False)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: user 23min 54s, sys: 1h 23min 10s, total: 1h 47min 5s
Wall time: 54min 38s


In [31]:
%%time
    
result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

CPU times: user 28.5 s, sys: 21.9 s, total: 50.4 s
Wall time: 25.9 s


#### Задание 4

In [32]:
def als_function(factors, regularization, iterations):
    print(f"Factors: {factors}")
    print(f"Regularization: {regularization}")
    print(f"Iterations: {iterations}")
    
    model = AlternatingLeastSquares(factors=factors, 
                                regularization=regularization,
                                iterations=iterations, 
                                calculate_training_loss=True, 
                                num_threads=10,
                                use_gpu=False)

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)
    
    result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

    score = result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()
    
    print(f"Score: {score}")

In [33]:
factors_variants = [32, 64, 128]
regularization_variants = [0.05, 0.1]
iterations_variants = [10, 15, 20]

In [ ]:
%%time

for factor in factors_variants:
    for regularization in regularization_variants:
        for iteration in iterations_variants:
            als_function(factor, regularization, iteration)

Factors: 32
Regularization: 0.05
Iterations: 10


  0%|          | 0/10 [00:00<?, ?it/s]

Score: 0.16875941737819966
Factors: 32
Regularization: 0.05
Iterations: 15


  0%|          | 0/15 [00:00<?, ?it/s]

Score: 0.16514314414866654
Factors: 32
Regularization: 0.05
Iterations: 20


...

  0%|          | 0/10 [00:00<?, ?it/s]

Score: 0.16042189854344327
Factors: 64
Regularization: 0.1
Iterations: 15


  0%|          | 0/15 [00:00<?, ?it/s]

Score: 0.15369161225514602
Factors: 64
Regularization: 0.1
Iterations: 20


  0%|          | 0/20 [00:00<?, ?it/s]

К сожалению выполнить код смог только в коллабе, так как локально не устанавливалась библиотека implicit версии 0.4.4 с которой всё работает. Но в коллабе слишком долго выполнялся код, более 10 часов и прервался.

Тем не менее  можно заметить что при возростании factors теряется score, да и при возростании остальных параметров он падает, но от factors зависит сильнее